In [43]:
# Importing the necessary dependencies
import pandas as pd
import numpy as np
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec
LabeledSentence = gensim.models.doc2vec.LabeledSentence

from keras.utils import np_utils

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Activation, LSTM, Embedding
from sklearn.model_selection import train_test_split

In [44]:
# get the data
# Convert the labels from str to int.
def getData():
    data = pd.read_csv("mydata.csv",error_bad_lines=False, encoding = "ISO-8859-1",header=None)
    data.columns = ["SentimentText","Sentiment"]
    data['Sentiment'] = data['Sentiment'].map(int)
    return data

data = getData()
pos = data[data.Sentiment == 1]
neg = data[data.Sentiment == 0]
print("Pos examples:",pos.shape,"Neg examples:", neg.shape)
data.head()

Pos examples: (27374, 2) Neg examples: (8344, 2)


,SentimentText,Sentiment
0,RT @Mayank_M_Joshi: Kovind will make an except...,1
1,@narendramodi @ashrafghani Hii pm modi good mo...,1
2,"And this "" love for Dalit "" emerged to sidelin...",1
3,RT @aijazzakasyed: @AijazZakaSyed writes in @S...,1
4,@soniandtv @vivekagnihotri Mamatajiððð...,1


In [45]:
# Tokenize the tweets
# Remove the tokens which start with '@' or 'http' or '#
def tokenize(tweet):
    try:
        tweet = str(tweet).lower()
        tokens = tokenizer.tokenize(tweet)
        tokens = [ token for token in tokens if not ( token.startswith("@") or token.startswith("http") or token.startswith("#")) ]
        return tokens
    except:
        return None

In [46]:
def postprocess(data):
    data['tokens'] = data['SentimentText'].progress_map(tokenize)
    return data

data = postprocess(data)



progress-bar:   0%|          | 0/35718 [00:00<?, ?it/s]

progress-bar:   5%|▍         | 1725/35718 [00:00<00:01, 17241.18it/s]

progress-bar:  10%|▉         | 3441/35718 [00:00<00:01, 17215.08it/s]

progress-bar:  15%|█▍        | 5225/35718 [00:00<00:01, 17397.12it/s]

progress-bar:  19%|█▉        | 6925/35718 [00:00<00:01, 17274.31it/s]

progress-bar:  24%|██▍       | 8703/35718 [00:00<00:01, 17422.96it/s]

progress-bar:  30%|██▉       | 10664/35718 [00:00<00:01, 18024.99it/s]

progress-bar:  35%|███▍      | 12422/35718 [00:00<00:01, 17887.80it/s]

progress-bar:  40%|████      | 14363/35718 [00:00<00:01, 18316.89it/s]

progress-bar:  45%|████▌     | 16101/35718 [00:00<00:01, 17966.13it/s]

progress-bar:  50%|████▉     | 17834/35718 [00:01<00:01, 17671.45it/s]

progress-bar:  55%|█████▍    | 19558/35718 [00:01<00:00, 17245.76it/s]

progress-bar:  60%|██████    | 21451/35718 [00:01<00:00, 17718.63it/s]

progress-bar:  65%|██████▍   | 23207/35718 [00:01<00:00, 17634.84it/s]

progress-b

In [47]:
# Break the data into test train split
n = data.shape[0]
x_train, x_test, y_train, y_test = train_test_split(np.array(data.head(n).tokens),
                                                    np.array(data.head(n).Sentiment), test_size=0.2)


In [48]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')



0it [00:00, ?it/s]

28574it [00:00, 437981.86it/s]

0it [00:00, ?it/s]

7144it [00:00, 466919.75it/s]

In [7]:
# Load the google's trained word3vec model
tweet_w2v = gensim.models.KeyedVectors.load_word2vec_format("dataset/google.bin.gz", binary=True)

In [49]:
# Converting a scentence to vector
# Basically averaging the vectors of different words
def wordvector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [50]:
from sklearn.preprocessing import scale
n_dim = 300
train_vecs_w2v = np.concatenate([wordvector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([wordvector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)



0it [00:00, ?it/s]

1792it [00:00, 17916.90it/s]

3550it [00:00, 17813.74it/s]

5335it [00:00, 17822.43it/s]

7077it [00:00, 17697.55it/s]

8892it [00:00, 17830.43it/s]

10644it [00:00, 17734.49it/s]

12379it [00:00, 17613.72it/s]

14144it [00:00, 17623.70it/s]

15913it [00:00, 17643.05it/s]

17651it [00:01, 17562.92it/s]

19391it [00:01, 17510.32it/s]

21149it [00:01, 17522.62it/s]

22911it [00:01, 17549.99it/s]

24683it [00:01, 17599.37it/s]

26432it [00:01, 17564.31it/s]

28181it [00:01, 17513.52it/s]

28574it [00:01, 17601.47it/s]

0it [00:00, ?it/s]

1749it [00:00, 17487.85it/s]

3499it [00:00, 17490.65it/s]

5253it [00:00, 17502.77it/s]

6943it [00:00, 17316.97it/s]

7144it [00:00, 17334.22it/s]

In [51]:
# Converting labels to one hot vector
# 0 -> [0,1]
# 1 - [1,0]
train_y = np_utils.to_categorical(y_train)
test_y = np_utils.to_categorical(y_test)

In [52]:
# building a model with 2 hidden layers
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=300, kernel_initializer='normal'))
model.add(Dense(512, activation='relu', kernel_initializer='normal'))
model.add(Dense(2, activation='softmax', kernel_initializer='normal'))
model.compile(optimizer=Adam(lr=0.0001),
              loss='mean_squared_error',
              metrics=['accuracy'])
model.fit(train_vecs_w2v, train_y, validation_data=(test_vecs_w2v, test_y), epochs=30, batch_size=100, verbose=1)

Train on 28574 samples, validate on 7144 samples
Epoch 1/30
28574/28574 [==============================] - 2s - loss: 0.0372 - acc: 0.9617 - val_loss: 0.0071 - val_acc: 0.9957
Epoch 2/30
28574/28574 [==============================] - 1s - loss: 0.0046 - acc: 0.9966 - val_loss: 0.0022 - val_acc: 0.9989
Epoch 3/30
28574/28574 [==============================] - 1s - loss: 0.0020 - acc: 0.9982 - val_loss: 0.0015 - val_acc: 0.9989
Epoch 4/30
28574/28574 [==============================] - 1s - loss: 0.0020 - acc: 0.9981 - val_loss: 0.0013 - val_acc: 0.9989
Epoch 5/30
28574/28574 [==============================] - 1s - loss: 0.0014 - acc: 0.9986 - val_loss: 2.7358e-04 - val_acc: 0.9999
Epoch 6/30
28574/28574 [==============================] - 1s - loss: 5.3568e-04 - acc: 0.9995 - val_loss: 2.0094e-04 - val_acc: 0.9999
Epoch 7/30
28574/28574 [==============================] - 1s - loss: 8.6296e-05 - acc: 1.0000 - val_loss: 1.7644e-04 - val_acc: 0.9999
Epoch 8/30
28574/28574 [==================

In [53]:
m = 108
print(data.SentimentText[m], data.Sentiment[m])
scentence = data.tokens[m]
scentence = tokenize("Kovind will make an exceptional president: PM Narendra Modi")
vec = wordvector(scentence, 300)
print(model.predict(vec))

To make the #InternationalYogaDay programme in #Lucknow a success, various NGOs and #Yoga gurus have camped there. https://t.co/TNuAucVF2Q 1
[[ 0.30909264  0.69090742]]


In [54]:
model.save("model1.h5")